# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous notebook, we have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this notebook, we will be performing more interactive visual analytics using `Folium`.

## Objectives


This lab contains the following tasks:

*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities


Import the required packages

In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

### Mark all launch sites on a map

First, let's add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, we can take a look at what are the coordinates for each site.

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


We create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We can use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate.

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [8]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    site, lat, long = row['Launch Site'], row['Lat'], row['Long']
    site_coord = [lat, long]
    folium.Circle(site_coord, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(site)).add_to(site_map)
    folium.map.Marker(site_coord, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site,)).add_to(site_map)
site_map

Now, we can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?


### Mark the successful/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.

The data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not.


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()

In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
color = []
for val in spacex_df['class']:
    if val==1:
        color.append('#008000')
    elif val==0:
        color.append('#FF0000')
spacex_df['marker_color']=color
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,#FF0000
1,CCAFS LC-40,28.562302,-80.577356,0,#FF0000
2,CCAFS LC-40,28.562302,-80.577356,0,#FF0000
3,CCAFS LC-40,28.562302,-80.577356,0,#FF0000
4,CCAFS LC-40,28.562302,-80.577356,0,#FF0000


For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    site, lat, long = record['Launch Site'], record['Lat'], record['Long']
    site_coord=[lat, long]
    folium.map.Marker(site_coord, icon=folium.Icon(color='white', icon_color=record['marker_color'], html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site,)).add_to(marker_cluster)

site_map

From the color-labeled markers in marker clusters, we can easily identify which launch sites have relatively high success rates.


### Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while exploring the map, we can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [14]:
from math import sin, cos, sqrt, atan2, radians

# Computes the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

#### Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

We consider the Launch Site CCAFS SLC-40 for the next tasks

In [15]:
# coordinates of the CCAFS SLC-40
launch_site_lat = 28.5632
launch_site_lon = -80.57685
site_coord=[launch_site_lat, launch_site_lon]

In [16]:
# coordinates of the closest coastline
coastline_lat = 28.56316
coastline_lon = -80.56796
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [17]:
coast_coord=[coastline_lat, coastline_lon]
folium.Marker(coast_coord,icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#D35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),)).add_to(site_map)

#### Draw a `PolyLine` between a launch site to the selected coastline point

#### BLUE line

In [18]:
coordinates=[site_coord, coast_coord]
lines=folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines)

Similarly, we can draw a line betwee a launch site to its closest city, railway, highway, etc. We must first use `MousePosition` to find their coordinates on the map first.

### Draw a PolyLine to the closest city

In [19]:
# coordinates of the closest city
city_lat = 28.07547
city_lon = -80.606
city_coord=[city_lat, city_lon]
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

In [20]:
folium.Marker(city_coord,icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#FF0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),)).add_to(site_map)

#### RED line

In [21]:
coordinates=[site_coord, city_coord]
lines=folium.PolyLine(locations=coordinates, weight=1, color='red')
site_map.add_child(lines)

### Draw a PolyLine to the closest railway

In [22]:
# coordinates of the closest railway
railway_lat = 28.50953
railway_lon = -80.78505
railway_coord=[railway_lat, railway_lon]
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

In [23]:
folium.Marker(railway_coord,icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#A020F0;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),)).add_to(site_map)

#### PURPLE line

In [24]:
coordinates=[site_coord, railway_coord]
lines=folium.PolyLine(locations=coordinates, weight=1, color='purple')
site_map.add_child(lines)

### Draw a PolyLine to the closest highway¶

In [25]:
# coordinates of the closest highway
highway_lat = 28.5634
highway_lon = -80.79844
highway_coord=[highway_lat, highway_lon]
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

In [26]:
folium.Marker(highway_coord,icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#196F3D ;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),)).add_to(site_map)

#### GREEN line

In [27]:
coordinates=[site_coord, highway_coord]
lines=folium.PolyLine(locations=coordinates, weight=1, color='green')
site_map.add_child(lines)

After ploting distance lines to the proximities, we can answer the following questions easily:

*   Are launch sites in close proximity to railways? No
*   Are launch sites in close proximity to highways? No
*   Are launch sites in close proximity to coastline? Yes
*   Do launch sites keep certain distance away from cities? Yes

# Next Steps:

Next, we will build a dashboard using Ploty Dash on detailed launch records.

## Author
### Diogo Miranda